# How Debuggers Work

In this chapter, we show how _interactive debuggers_ work – tools that allow you to observe the program state during an execution.  Thanks to the power of Python, we can even build our own debugger in a few lines of code.

**Prerequisites**

* You should have read the [Introduction to Debugging](Intro_Debugging.ipynb).
* Knowing a bit of _Python_ is helpful for understanding the code examples in the book.

In [1]:
import bookutils

/Users/zeller/Library/Python/3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Debugger Features

_Interactive Debuggers_ (or short *debuggers*) are tools that allow you to observe program executions. A debugger typically offers the following features:

* _Run_ the program
* Define a condition under which the execution should _stop_ and hand over control to the debugger. Conditions include
    * a particular location is reached
    * a particular variable takes a particular value
    * or some other condition of choice.
* When the program stops, you can _observe_ the current state, including
    * the current location
    * variables and their values
    * the current function and its callers
* When the program stops, you can _step_ through program execution, having it stop at the next instruction again.
* Finally, you can also _resume_ execution to the next stop.

These commands typically are used in a loop. First, you identify the location(s) you want to inspect, and tell the debugger to stop execution once one of these location(s) is reached. Then you have the debugger run the program. When it stops at the given location, you inspect the state (and check whether things are as expected). You can then step through the program or define new stop conditions and resume execution.

This functionality can come as a _command-line interface_, typing commands at a prompt, or as a _graphical user interface_, selecting commands from the screen. Debuggers can come as standalone tools, or be integrated into a programming environment of choice.

## Debugger Interaction

The key idea of an _interactive_ debugger is to set up the tracing function such that it actually _asks_ what to do next, prompting you to enter a _command_. For the sake of simplicity, we collect such a command from a command line, using the Python `input()` function. The following code prompts you to enter a command:

In [2]:
INPUTS = []

In [3]:
from bookutils import HTML

In [4]:
def my_input(prompt):
    given_input = None
    try:
        global INPUTS
        given_input = INPUTS[0]
        INPUTS = INPUTS[1:]
    except:
        pass
    
    if given_input:
        display(HTML(f"<pre>{prompt}{given_input}</pre>"))
        return given_input
    
    return input(prompt)

In [5]:
from Intro_Debugging import remove_html_markup

In [6]:
import inspect

In [7]:
from Tracer import Tracer

In [8]:
class Debugger(Tracer):
    def __init__(self):
        self.stepping = True
        self.breakpoints = set()
        self.interact = True
        
        self.frame = None
        self.event = None
        self.arg = None

        super().__init__()

    def step_command(self, arg=""):
        """Execute up to the next line"""
        self.stepping = True
        self.interact = False

    def continue_command(self, arg=""):
        """Resume execution"""
        self.stepping = False
        self.interact = False

    def list_command(self, arg=""):
        """Show current function. If function is given, show its source code."""
        if arg:
            try:
                obj = eval(arg)
                source_lines, line_number = inspect.getsourcelines(obj)
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")
                return
            current_line = -1
        else:
            source_lines, line_number = inspect.getsourcelines(self.frame.f_code)
            current_line = self.frame.f_lineno
    
        for line in source_lines:
            spacer = ' '
            if line_number == current_line:
                spacer = '>'
            elif line_number in self.breakpoints:
                spacer = '#'
            self.log(f'{line_number:4}{spacer} {line}', end='')
            line_number += 1
    
    def print_command(self, arg=""):
        """Print an expression. If no expression is given, print all variables"""
        vars = self.frame.f_locals
        
        if not arg:
            self.log("\n".join([f"{var} = {repr(vars[var])}" for var in vars]))
        else:
            try:
                self.log(f"{arg} = {repr(eval(arg, globals(), vars))}")
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")

    def break_command(self, arg=""):
        """Set a breakoint in given line. If no line is given, print all breakpoints"""
        if arg:
            self.breakpoints.add(int(arg))
        self.log("Breakpoints:", self.breakpoints)

    def delete_command(self, arg=""):
        """Delete a breakoint in given line. If no line is given, clear all breakpoints"""
        if arg:
            try:
                self.breakpoints.remove(int(arg))
            except KeyError:
                self.log(f"No such breakpoint: {arg}")
        else:
            self.breakpoints = []
        self.log("Breakpoints:", self.breakpoints)
        
    def quit_command(self, arg=""):
        """Finish execution"""
        self.breakpoints = []
        self.stepping = False
        self.interact = False
        
    def help_command(self, command=""):
        """Give help on given command. If no command is given, give help on all"""
        
        if command:
            possible_cmds = [possible_cmd for possible_cmd in self.commands() if possible_cmd.startswith(command)]

            if len(possible_cmds) == 0:
                self.log(f"Unknown command {repr(command)}. Possible commands are:")
                possible_cmds = self.commands()
            elif len(possible_cmds) > 1:
                self.log(f"Ambiguous command {repr(command)}. Possible expansions are:")
                
        else:
            possible_cmds = self.commands()

        for cmd in possible_cmds:
            method = self.command_method(cmd)
            self.log(f"{cmd:10} -- {method.__doc__}")

    def command_method(self, command):
        if command.startswith('#'):
            return None

        possible_cmds = [possible_cmd for possible_cmd in self.commands() if possible_cmd.startswith(command)]
        if len(possible_cmds) != 1:
            self.help_command(command)
            return None
        
        cmd = possible_cmds[0]
        return getattr(self, cmd + '_command')
        
    def execute(self, command):
        sep = command.find(' ')
        if sep > 0:
            cmd = command[:sep].strip()
            arg = command[sep + 1:].strip()
        else:
            cmd = command.strip()
            arg = ""

        method = self.command_method(cmd)
        if method:
            method(arg)

    def commands(self):
        cmds = [method.replace('_command', '') for method in dir(self.__class__) if method.endswith('_command')]
        cmds.sort()
        return cmds
    
    def traceit(self, frame, event, arg):
        self.frame = frame
        self.event = event
        self.arg = arg

        if self.stepping or frame.f_lineno in self.breakpoints:
            self.print_debugger_status(frame, event, arg)

            self.interact = True
            while self.interact:
                command = my_input("(debugger) ")
                self.execute(command)

        return self.traceit

In [9]:
debugger = Debugger()

In [10]:
debugger.commands()

['break', 'continue', 'delete', 'help', 'list', 'print', 'quit', 'step']

In [11]:
debugger.execute('h')

break      -- Set a breakoint in given line. If no line is given, print all breakpoints
continue   -- Resume execution
delete     -- Delete a breakoint in given line. If no line is given, clear all breakpoints
help       -- Give help on given command. If no command is given, give help on all
list       -- Show current function. If function is given, show its source code.
print      -- Print an expression. If no expression is given, print all variables
quit       -- Finish execution
step       -- Execute up to the next line


In [12]:
debugger.execute('b 25')

Breakpoints: {25}


In [13]:
debugger.execute('h q')

quit       -- Finish execution


In [14]:
debugger.execute('co')

In [15]:
with Debugger():
    ret = remove_html_markup("<b>abc</b>")
ret

Calling remove_html_markup(s = '<b>abc</b>')


(debugger)  q


'abc'

## Synopsis

This chapter provides a `Tracer()` class that allows to log events during program execution.

The advanced subclass `EventTracer` allows to restrict logs to specific conditions. Logs are shown only while the given `condition` holds:

It also allows to restrict logs to specific events. Log entries are shown only if one of the given `events` changes its value:

## Lessons Learned

* Interpreted languages can provide _debugging hooks_ that allow to dynamically control program execution and access program state.
* Tracing can be limited to specific conditions and events:
    * A _breakpoint_ is a condition referring to a particular location in the code.
    * A _watchpoint_ is an event referring to a particular state change.
* Compiled languages allow to _instrument_ code at compile time, injecting code that allows to hand over control to a tracing or debugging tool.

## Next Steps

In the next chapter, we will see how to

* [leverage our tracing infrastructure for interactive debugger](Debugger.ipynb)


## Background

\todo{add}

## Exercises


### Exercise 1: More Commands

Extending the `Debugger` class with extra features and commands is a breeze. The following commands are inspired from [the GNU command-line debugger (GDB)](https://www.gnu.org/software/gdb/):

#### Step over functions ("next")

When stopped at a function call, the `next` command should execute the entire call, stopping when the function returns. (In contrast, `step` stops at the first line of the function called.)

#### Print call stack ("where")

Implement a `where` command that shows the stack of calling functions.

#### Move up and down the call stack ("up" and "down")

After entering the `up` command, explore the source and variables of the _calling_ function rather than the current function. Use `up` repeatedly to move further up the stack. `down` returns to the caller.

#### Named breakpoints ("break")

With `break FUNCTION` and `delete FUNCTION`, set and delete a breakpoint at `FUNCTION`.

#### Execute until line ("until")

With `until LINE`, resume execution until a line greater than `LINE` is reached. If `LINE` is not given, resume execution until a line greater than the current is reached. This is useful to avoid stepping through multiple loop iterations.

#### Execute until return ("finish")

With `finish`, resume execution until the current fucnction returns.

#### Watchpoints ("watch")

With `watch CONDITION`, stop execution as soon as `CONDITION` changes its value. (Use the code from our `EventTracer` class in the [chapter on Tracing](Tracer.ipynb).) `delete CONDITION` removes the watchpoint.

### Exercise 2: Changing State

Some Python implementations allow to alter the state, by assigning values to `frame.f_locals`. Implement a `set VAR VALUE` command that allows to change the value of (local) variable `VAR` to the new value `VALUE`.

### Exercise 3: Time-Travel Debugging

Rather than inspecting a function at the moment it executes, you can also _record_ the entire state (call stack, local variables, etc.) during execution, and then run an interactive session to step through the recorded execution. This allows for a `back` command which gets you back to earlier states.